In [7]:
import os
import torch
from enum import Enum

In [8]:
class Scheme(Enum):
    DIRECTED_6W = 1
    UNDIRECTED_6W = 2
    DIRECTED_1W = 3
    UNDIRECTED_1W = 4

In [9]:
model_name = 'google-t5/t5-large'
max_answer_length = 300
BATCH_SIZE = 10
accumulate_step = None
DATA_NAME_SINGLE = 'obqa_fact'
DATABASE_TEST_SINGLE = None
NUM_EPOCHS = 1
VISIBLE_DEVICE ='0,1' #','.join([str(x) for x in range(torch.cuda.device_count())])
DEVICE = 0
STORE_MODE = 'loss_only'  # loss_only
SET_MODE = 'new'
EVALUATE = True
USE_16_bit = False
SCHEME=Scheme.UNDIRECTED_6W
MAX_MEMORY = None  # max_memory=

In [10]:
if not DATABASE_TEST_SINGLE:
    DATABASE_TEST_SINGLE = DATA_NAME_SINGLE
os.environ['CUDA_VISIBLE_DEVICES'] = VISIBLE_DEVICE  # cannot work
print(f"Loi params: {model_name}{'#'*5}{max_answer_length}{'#'*5}batch_size={BATCH_SIZE};accumulate_step={accumulate_step}")
DATABASE_NAME = [DATA_NAME_SINGLE]  # ,'arc_hard']#,'race','mctest',]
DATABASE_TESTS = [DATABASE_TEST_SINGLE]

Loi params: google-t5/t5-large#####300#####batch_size=10;accumulate_step=None


In [11]:
import torch
import pickle
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import trange
from transformers import T5Tokenizer, T5ForConditionalGeneration, Adafactor
from functools import wraps, partial
from torch.nn.modules.sparse import Embedding
from torch.optim import Adam, SGD
import torch.nn as nn
from tqdm.notebook import tqdm
from random_utils import set_seed
import pandas as pd

In [12]:
# model_name = "loi_with_padding_just_same_answer_len_70.pkl"

# "loi_with_padding_1.pkl"#
# model_name = (
#     "allenai/unifiedqa-v2-t5-large-1363200"  # you can specify the model size here
# )
tokenizer = T5Tokenizer.from_pretrained(model_name)
kwargs = {}
if USE_16_bit:
    kwargs['torch_dtype'] = torch.float16
kwargs['device_map'] = 'auto'
if MAX_MEMORY:
    max_memory = {0: "5GB", 1: "5GB", 2: "5GB", "cpu": "504GB"}
model_original = T5ForConditionalGeneration.from_pretrained(
    model_name, **kwargs)  # 'auto')
model = model_original
# model.hf_device_map

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [13]:

dataset_test = [[] for x in DATABASE_TESTS]
dataset_train = []
moi_index = []
if EVALUATE:
    for i, dataname in enumerate(DATABASE_TESTS):
        dataset_test[i] = pickle.load(
            open(f"multiple_choice_datasets/{dataname}_test.pkl", "rb"))
        print(dataset_test[i])

for i, dataname in enumerate(DATABASE_NAME):
    moi_index.append(len(dataset_train))
    dataset_train.extend(pickle.load(
        open(f"multiple_choice_datasets/{dataname}_train.pkl", "rb")))
datas = dataset_train
for i in range(2):
    print(f""" max: {max([len(x[i]) for x in datas])} 
        min:{min([len(x[i]) for x in datas])}
            avg:{sum([len(x[i]) for x in datas])/len(datas)}
            len: {len(datas)}""")
if EVALUATE:
    datas = dataset_test[0]
    for i in range(2):
        print(f""" max: {max([len(x[i]) for x in datas])} 
                min:{min([len(x[i]) for x in datas])}
                    avg:{sum([len(x[i]) for x in datas])/len(datas)}
                    len: {len(datas)}""")

[('Context: using less resources usually causes money to be saved. Question: A person wants to start saving money so that they can afford a nice vacation at the end of the year. After looking over their budget and expenses, they decide the best way to save money is to \\n ( ) make more phone calls ( ) quit eating lunch out ( ) buy less with monopoly money ( ) have lunch with friends', 'quit eating lunch out'), ('Context: fog is formed by water vapor condensing in the air. Question: There is most likely going to be fog around: \\n ( ) a marsh ( ) a tundra ( ) the plains ( ) a desert', 'a marsh'), ('Context: predators eat prey. Question: Predators eat \\n ( ) lions ( ) humans ( ) bunnies ( ) grass', 'bunnies'), ('Context: if a tree is located near a sidewalk then the roots of that tree might crack the sidewalk. Question: Oak tree seeds are planted and a sidewalk is paved right next to that spot, until eventually, the tree is tall and the roots must extend past the sidewalk, which means \

In [14]:
def hook(hook_before, oldfunc, hook_after):

    def foo(*args, **kwargs):
        hook_before(*args, **kwargs)
        aa = oldfunc(*args, **kwargs)
        hook_after(*args, **kwargs)
        return aa

    return foo

In [15]:
def new_compute_bias(self, query_length, key_length, device=None):
    """Compute binned relative position bias"""
    if device is None:
        device = self.relative_attention_bias.weight.device
    context_position = torch.arange(query_length, dtype=torch.long, device=device)[
        :, None
    ]
    memory_position = torch.arange(
        key_length, dtype=torch.long, device=device)[None, :]

    relative_position = (
        memory_position - context_position
    )  # shape (query_length, key_length)
    if self.is_decoder:
        values = values.permute([2, 0, 1]).unsqueeze(
            0
        )  # shape (1, num_heads, query_length, key_length)
        return values

    anchors = self.anchor_array
    values = []
    for anchor in anchors:
        mot = [[anchor[idx], anchor[idx+1]] for idx in range(len(anchor)-1)]
        max_answer_length = max([x[1] - x[0] for x in mot])
        # print(a, b, c, d, max_answer_length)
        context_position_new = context_position.clone()
        for i in range(1, len(mot)):
            context_position_new[mot[i][0]: mot[i][1]
                                 ] = context_position_new[mot[i][0]: mot[i][1]] - mot[0][0]
        context_position_new[-1] = mot[0][0] + 2 * max_answer_length
        memory_position_new = context_position_new.clone().view(1, -1)
        relative_position = (
            memory_position_new - context_position_new
        )  # shape (query_length, key_length)
        for i in range(len(mot)):
            for j in range(len(mot)):
                if i != j:
                    x = mot[i]
                    y = mot[j]
                    relative_position[x[0]: x[1], y[0]
                        : y[1]] += max_answer_length
        relative_position_bucket = self._relative_position_bucket(
            relative_position,  # shape (query_length, key_length)
            bidirectional=(not self.is_decoder),
            num_buckets=self.relative_attention_num_buckets,
            max_distance=self.relative_attention_max_distance,
        )
        value = self.relative_attention_bias(relative_position_bucket)
        
        DEC = {"01": 0, "02": 1, "03": 2, "12": 3, "13": 4, "23": 5}
        six_mask_turn_off = torch.ones((value.shape[0], value.shape[1], 16))
        six_mask_turn_on_forward = torch.zeros((6, value.shape[0],value.shape[1], 16))
        six_mask_turn_on_backward = torch.zeros((6, value.shape[0],value.shape[1], 16))
        for i in range(len(mot) - 1):
            for j in range(i + 1, len(mot)):
                x = mot[i]
                y = mot[j]
                comb_index = DEC[f"{i}{j}"]
                six_mask_turn_on_forward[comb_index, x[0] : x[1], y[0] : y[1], :] = 1
                six_mask_turn_on_backward[comb_index, y[0] : y[1], x[0] : x[1], :] = 1
                six_mask_turn_off[x[0] : x[1], y[0] : y[1], :] = 0
                six_mask_turn_off[x[0] : x[1], y[0] : y[1], :] = 0

        device_of_values = value.device
        six_extra_embedding_forward = [
            self.extra_dimension_embedding_forward[i](torch.tensor(
                [0])).view(1, 1, 16).to(device_of_values) for i in range(6)
        ]
        six_extra_embedding_backward = [
            self.extra_dimension_embedding_backward[i](torch.tensor(
                [0])).view(1, 1, 16).to(device_of_values) for i in range(6)
        ]
        tmp = torch.zeros_like(value).to(device_of_values)
        for i in range(len(six_extra_embedding_forward)):
            if SCHEME==Scheme.DIRECTED_6W:
                tmp += (six_mask_turn_on_forward[i].to(device_of_values) *
                        six_extra_embedding_forward[i] +
                        six_mask_turn_on_backward[i].to(device_of_values) *
                        six_extra_embedding_backward[i])
            elif SCHEME==Scheme.UNDIRECTED_6W:
                tmp += (six_mask_turn_on_forward[i].to(device_of_values) *
                        six_extra_embedding_forward[i] +
                        six_mask_turn_on_backward[i].to(device_of_values) *
                        six_extra_embedding_forward[i])
            elif SCHEME==Scheme.DIRECTED_1W:
                tmp += (six_mask_turn_on_forward[0].to(device_of_values) *
                        six_extra_embedding_forward[i] +
                        six_mask_turn_on_backward[0].to(device_of_values) *
                        six_extra_embedding_forward[i])
            elif SCHEME==Scheme.UNDIRECTED_1W:
                tmp += (six_mask_turn_on_forward[0].to(device_of_values) *
                        six_extra_embedding_forward[0] +
                        six_mask_turn_on_backward[0].to(device_of_values) *
                        six_extra_embedding_forward[0])
        value = value * six_mask_turn_off.to(device_of_values) + tmp
        values.append(value)
    values = torch.stack(values)  # shape [1, 91, 91, 16]
    values = values.permute(
        [0, 3, 1, 2]
    )  # shape (batch size, num_heads, query_length, key_length)
    return values

In [16]:

import textwrap


extra_dim_learning = []


def set_mode(MODE):
    itself = model.encoder.block[0].layer[0].SelfAttention
    if MODE == "new":
        # itself.forward = partial(modified_self_attention_forward, itself)
        itself.compute_bias = partial(new_compute_bias, itself)
        model.forward = hook(input_before_hooker, partial(
            T5ForConditionalGeneration.forward, model), input_after_hooker)
        tmp_extra_dim_learning = [[], []]
        for i in range(2):
            for _ in range(6):
                new_emb = Embedding(1, 16)
                new_emb.weight.data.normal_(mean=0.0, std=1024**-0.5)
                tmp_extra_dim_learning[i].append(new_emb)

        itself.extra_dimension_embedding_forward = nn.ModuleList(
            tmp_extra_dim_learning[0]
        )
        itself.extra_dimension_embedding_backward = nn.ModuleList(
            tmp_extra_dim_learning[1]
        )
    else:
        # itself.forward = partial(
        #     model.encoder.block[0].layer[0].SelfAttention.__class__.forward, itself
        # )
        itself.compute_bias = partial(
            model.encoder.block[0].layer[0].SelfAttention.__class__.compute_bias, itself
        )
        model.forward = partial(T5ForConditionalGeneration.forward, model)


def check_encoded(all_input_ids):
    anchors = []
    for input_ids in all_input_ids:
        # print('\n'.join([f'{x.item()},{y}' for x,y in zip(input_ids, tokens)][50:]))
        original = input_ids.tolist()
        anchor = []
        for i in range(len(input_ids)):
            if (
                i < len(input_ids) - 2
                and input_ids[i] == 41
                and input_ids[i + 1] == 3
                and input_ids[i + 2] == 61
            ) or original[i] == 1:
                anchor.append(i)
        anchors.append(anchor)
    return anchors


def input_before_hooker(*args, **kwargs):
    input_ids = kwargs["input_ids"]
    # print('old ',input_ids)
    anchors = check_encoded(input_ids)
    final_inputs = []
    for input_id, anchor in zip(input_ids, anchors):
        input_id = input_id.tolist()

        real_max_len = max([anchor[idx+1] - anchor[idx]
                           for idx in range(len(anchor)-1)])
        if real_max_len > max_answer_length:
            print(f'ALERT: MAX LENGTH IS {real_max_len}')
        for x in reversed(range(1, len(anchor))):
            if anchor[x] - anchor[x - 1] < max_answer_length:
                [
                    input_id.insert(anchor[x], 0)
                    for _ in range(max_answer_length - (anchor[x] - anchor[x - 1]))
                ]

        final_inputs.append(input_id)

    max_length = max([len(input) for input in final_inputs])
    mask = [[1]*max_length]*len(final_inputs)
    for idx, input in enumerate(final_inputs):
        for x in range(max_length):
            if x >= len(input):
                mask[idx][x] = 0
        for x in range(max_length-len(input)):
            input.append(0)
    kwargs["input_ids"] = torch.tensor(final_inputs).to(input_ids.device)
    kwargs['attention_mask'] = torch.tensor(mask).to(input_ids.device)
    # print('new ',kwargs["input_ids"])
    anchors = check_encoded(kwargs["input_ids"])
    model.encoder.block[0].layer[0].SelfAttention.anchor_array = anchors


def input_after_hooker(*args, **kwargs):
    model.encoder.block[0].layer[0].SelfAttention.anchor_array = None


print(textwrap.fill(dataset_train[0][0]))

Context: the sun is the source of energy for physical cycles on Earth.
Question: The sun is responsible for \n ( ) puppies learning new
tricks ( ) children growing up and getting old ( ) flowers wilting in
a vase ( ) plants sprouting, blooming and wilting


In [17]:

def measure_unalike(arr, print_arr=False):
    n = len(arr)
    arr = pd.Series(arr).value_counts()
    if print_arr:
        print(arr)
    return 1 - ((arr / n)**2).sum()


question_to_do = 5
per_question = 20


def get_model_forward(input_ids, attention_mask, model=model):
    with torch.no_grad():
        start = []
        [start.append([0]) for x in range(len(input_ids))]
        for k in range(max_answer_length):
            # print(torch.tensor(start).shape)
            result = model(
                input_ids=input_ids.to(DEVICE),
                attention_mask=attention_mask.to(DEVICE),
                decoder_input_ids=torch.tensor(start).to(DEVICE),
                output_attentions=True,
            )
            item = result.logits.argmax(dim=2)[:, -1]
            # print('loi',result.logits.shape, item)
            for index in range(len(item)):
                start[index].append(item[index].item())
            if torch.allclose(item, torch.tensor(1)):
                break
            #     break
    result = []
    for batch in start:
        y = -1
        for index, x in enumerate(batch):
            if x == 1:
                y = index
                break
        result.append(batch[:y+1] if y > -1 else batch)
    return [tokenizer.decode(x, skip_special_tokens=True) for x in result]


def run_model(input_strs):
    if input_strs is str:
        input_strs = [input_strs]
    input_ids_wrapper = tokenizer(
        input_strs, padding=True, return_tensors='pt')

    answer = get_model_forward(input_ids_wrapper['input_ids'],
                               input_ids_wrapper['attention_mask'])
    return answer


set_mode(SET_MODE)
# print(SET_MODE, run_model(dataset_train[0][0]))

In [18]:
model.encoder.block[0].layer[0].SelfAttention

T5Attention(
  (q): Linear(in_features=1024, out_features=1024, bias=False)
  (k): Linear(in_features=1024, out_features=1024, bias=False)
  (v): Linear(in_features=1024, out_features=1024, bias=False)
  (o): Linear(in_features=1024, out_features=1024, bias=False)
  (relative_attention_bias): Embedding(32, 16)
  (extra_dimension_embedding_forward): ModuleList(
    (0-5): 6 x Embedding(1, 16)
  )
  (extra_dimension_embedding_backward): ModuleList(
    (0-5): 6 x Embedding(1, 16)
  )
)

In [19]:
# l1=[x.cpu().detach().numpy() for x in model.encoder.block[0].layer[0].SelfAttention.relative_attention_bias.weight]
# l2=[x.weight.detach().numpy() for x in model.encoder.block[0].layer[0].SelfAttention.extra_dimension_embedding_forward]
# pickle.dump((l1,l2),open(f'Scheme.Undirected_6w_weight_before.pkl','wb'))

In [20]:
kk = [(index, x, y) for index, (x, y) in enumerate(model.named_parameters())
      if y.requires_grad == True]
[(index, x) for index, x, y in kk if "decoder" in x]
len(kk)
all_position_weight = [
    y for index, x, y in kk if ("extra_dimension_embedding" in x) or (
        ("encoder" in x) and ("relative_attention_bias" in x))
]
to_train_model = [y for index, x, y in kk]

In [21]:
data_array = [
    (ques, answer, ques.split(" ( ) ")[1:])
    for ques, answer in [
        (
            dataset_train[x][0],
            dataset_train[x][1],
        )
        for x in range(len(dataset_train))
    ]
]

In [22]:
# [(print(idx),CheckTransform.__call__(None, data_array[idx])) for idx,x in enumerate(data_array)]

In [23]:
class CheckTransform(object):

    def __call__(self, sample):
        # print(f"'{sample[1]}'")
        try:
            return {
                "input_ids": sample[0],
                "label_index": sample[2].index(sample[1]),
                "all_labels": sample[2],
            }
        except:
            raise Exception(f'ERror: {sample}')
            print('all answer: ', sample[2])
            print('answer', sample[1])


class CustomDataset(Dataset):

    def __init__(self, dataset_array, transform=None):
        self.dataset = dataset_array
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        return self.transform(self.dataset[idx])


def collate(datas):
    wrapper = tokenizer([x["all_labels"][x['label_index']] for x in datas],
                        padding=True)
    wrapper["label_ids"] = torch.tensor(wrapper.pop("input_ids"))
    # wrapper["label_index"] = torch.tensor([x["label_index"] for x in datas])
    for k in wrapper["label_ids"]:
        k[k == tokenizer.pad_token_id] = -100
    wrapper["all_decoder_attention_masks"] = torch.tensor(
        wrapper.pop("attention_mask"))
    for_input = tokenizer([x["input_ids"] for x in datas], padding=True)
    wrapper['input_ids'] = torch.tensor(for_input.pop('input_ids'))
    wrapper['attention_mask'] = torch.tensor(for_input.pop('attention_mask'))
    return wrapper


loi_dataloader = DataLoader(
    CustomDataset(
        data_array,
        CheckTransform(),
    ),
    batch_size=1 if accumulate_step else BATCH_SIZE,
    shuffle=True,
    collate_fn=collate,
)
# for k in loi_dataloader:
#     print(k["all_label_ids"])
#     break

In [24]:
from transformers.optimization import Adafactor, AdafactorSchedule

# attention 898704
# hidden state 242688
# classification_layer = nn.Linear(242688, 4).to(DEVICE)
# optimizer =AdamW(lr=1e-4)
optimizer = Adafactor(
    to_train_model,  # + [x for x in classification_layer.parameters()],
    relative_step=False,
    lr=1e-4,
)
# lr_schedule=AdafactorSchedule(optimizer)


def evaluate():
    ll = 1
    wrong_answers = [[] for x in range(ll)]
    got_2 = [[] for x in range(ll)]
    got_1 = [[] for x in range(ll)]
    answers = [[] for x in range(ll)]
    last_str = None
    last_acc = None
    for ix in range(ll):
        # print(f'Name {DATABASE_TESTS[ix]}')
        # [pickle.load(open(f"multiple_choice_datasets/obqa_fact_test.pkl", "rb"))]:
        # print(f"test {data==dataset_test}")
        count = 0
        count1 = 0
        count2 = 0
        count10 = 0
        total = 0
        data = dataset_test[ix]
        pbar1 = tqdm(range(len(data)))
        for ques in pbar1:
            question = data[ques][0]
            key = data[ques][1]
            total += 1
            answer = run_model(question)[0]
            answers[ix].append(answer)
            if key == answer:
                count += 1
            else:
                wrong_answers[ix].append(ques)
            if key[0] == answer[0]:
                count1 += 1
                got_1[ix].append(ques)
            if key[:2] == answer[:2]:
                count2 += 1
                got_2[ix].append(ques)
            if answer in question:
                count10 += 1
            last_str = f"{count}, {count1}, {count2}, {count10},{total},{count/total*100:.2f},{count10/total*100:.2f}"
            last_acc = f'{count/total*100:.2f}'
            pbar1.set_postfix_str(last_str)
    return last_str, last_acc

In [25]:
# def turn_position_learning(on):
#     for x in all_position_weight:
#         x.requires_grad = on


loss_running_score = 0
correct_running_score = 0
conform_running_score = 0
count = 0
extra_info = ""
res_tokens = []
optimizer.zero_grad()
set_seed(42)
turn_position = False
# turn_position_learning(False)
evaluates = []
loss_array = []
for learn_pos in range(NUM_EPOCHS):
    pbar = tqdm(loi_dataloader)
    for wrapper in pbar:
        count += 1
        result = model(
            input_ids=wrapper["input_ids"].to(DEVICE),
            attention_mask=wrapper['attention_mask'].to(DEVICE),
            labels=wrapper['label_ids'].to(DEVICE),
            decoder_attention_mask=wrapper["all_decoder_attention_masks"].to(
                DEVICE),  # output_attentions=True
        )
        loss = result.loss
        if accumulate_step:
            loss = loss/accumulate_step
        loss.backward()
        if accumulate_step:
            if count % accumulate_step == 0:
                optimizer.step()
                optimizer.zero_grad()
                loss_running_score = loss_running_score * 0.9 + result.loss.item() * 0.1
        else:
            loss_running_score = loss_running_score * 0.9 + loss.item() * 0.1
            optimizer.step()
            optimizer.zero_grad()
        extra_info=''
        if EVALUATE:
            with torch.no_grad():
                if count%150==0:
                    all_evaluate_infos = evaluate()
                    print('acc: ',all_evaluate_infos)
                # if count % 100 == 0:
                #     extra_info = run_model(dataset_test[0][0][0])
        pbar.set_description_str(f"Loss: {loss_running_score:.3f}")
        pbar.set_postfix_str(extra_info)
        loss_array.append(loss_running_score)
    path_split = model_name.split('/')
    prefix = f"{path_split[0]}/{SCHEME}_{SET_MODE}_{path_split[1]}_{DATA_NAME_SINGLE}_epoch_{learn_pos}"
    if STORE_MODE == 'loss_only':
        pickle.dump(loss_array, open(f"{prefix}_loss.pkl", 'wb'))
    else:
        eval_str = ''
        if EVALUATE:
            all_evaluate_infos = evaluate()
            evaluates.append(
                f'{all_evaluate_infos[0]}-Loss:{loss_running_score}')
            eval_str = f'_eval_{DATABASE_TEST_SINGLE}_acc_{all_evaluate_infos[1]}'
        model.save_pretrained(f"{prefix}_{eval_str}.pkl", from_pt=True)
    # before_lr = optimizer.param_groups[0]["lr"]
    # after_lr = optimizer.param_groups[0]["lr"]
    # print("Epoch %d: SGD lr %.4f -> %.4f" % (learn_pos, before_lr, after_lr))
print(evaluates)

  0%|          | 0/496 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
print(prefix,eval_str)

### Measure resilient

In [ ]:
def measure_unalike(arr):
    n = len(arr)
    arr = pd.Series(arr).value_counts()
    return 1 - ((arr / n)**2).sum()


measure_unalike(["a", "a", "a"])